# 📘 Seção 1: Instalações e Imports

In [1]:
# Instale as dependências necessárias
import requests
import pandas as pd
from datetime import datetime, timedelta
import pytz
import os
import plotly.express as px
import glob

# 🔐 Seção 2: Configuração do Endpoint da The Graph

In [2]:
API_KEY = "54b3ae486458761ae740097bf7665918"

GRAPHQL_URL = f"https://gateway.thegraph.com/api/{API_KEY}/subgraphs/id/5zvR82QoaXYFyDEKLZ9t6v9adgnptxYpKpSbxtgVENFV"

# 📡 Seção 3: Consulta das 5 maiores pools de liquidez

In [3]:
def get_top_5_pools():
    query = {
        "query": """
        {
          pools(orderBy: volumeUSD, orderDirection: desc, first: 5) {
            id,
            volumeUSD,
            feeTier,
            token0Price,
            token1Price,
            liquidity,
            token0 {
              symbol,
              id,
              decimals
            }
            token1 {
              symbol,
              id,
              decimals
            }
          }
        }
        """
    }
    response = requests.post(GRAPHQL_URL, json=query)
    response.raise_for_status()
    data = response.json()
    if "errors" in data:
        raise Exception(f"GraphQL error: {data['errors']}")
    return data["data"]["pools"]

# 📅 Seção 4 Buscar dados agregados dos últimos 30 dias para uma pool

In [4]:
def get_unix_timestamp_days_ago(days: int) -> int:
    dt = datetime.now(pytz.utc) - timedelta(days=days)
    return int(dt.timestamp())

def build_day_data_query(pool_id, start_timestamp):
    return {
        "query": f"""
        {{
          poolDayDatas(
            first: 1000,
            orderBy: date,
            orderDirection: desc,
            where: {{
              pool: "{pool_id}",
              date_gte: {start_timestamp}
            }}
          ) {{
            date
            volumeUSD
            tvlUSD
            feesUSD
            token0Price
            token1Price
            liquidity
          }}
        }}
        """
    }

def fetch_pool_day_data(pool_id):
    start_timestamp = get_unix_timestamp_days_ago(90)
    query = build_day_data_query(pool_id, start_timestamp)
    try:
        response = requests.post(GRAPHQL_URL, json=query)
        response.raise_for_status()
        data = response.json()
        if "errors" in data:
            raise Exception(f"GraphQL error: {data['errors']}")
        return data["data"]["poolDayDatas"]
    except requests.RequestException as e:
        print("Erro de requisição:", e)
    except Exception as e:
        print("Erro na resposta ou estrutura da API:", e)
    return []

# 🧮 Seção 5. Processar os dados em DataFrame

In [5]:
def process_data(raw_data, fee_tier):
    if not raw_data:
        return pd.DataFrame()

    df = pd.DataFrame(raw_data)
    df['date'] = pd.to_datetime(df['date'], unit='s')

    numeric_cols = ['volumeUSD', 'tvlUSD', 'feesUSD', 'token0Price', 'token1Price', 'liquidity']
    for col in numeric_cols:
        df[col] = pd.to_numeric(df[col], errors='coerce')

    df = df.sort_values("date")
    df['feeTier'] = fee_tier / 10000

    # Cálculo de APR diário (sem composição)
    df['dailyAPR'] = df.apply(
        lambda row: ((row['feesUSD'] / row['tvlUSD']) * 365 * 100) if row['tvlUSD'] > 0 else 0,
        axis=1
    )

    return df

# 💾 Seção 6. Executar tudo e salvar em CSV

In [6]:
output_dir = "pool_data_csv"
os.makedirs(output_dir, exist_ok=True)
top_pools = get_top_5_pools()
print("🔝 Top 5 Pools por volumeUSD:\n")

for i, pool in enumerate(top_pools):
    token0 = pool['token0']['symbol']
    token1 = pool['token1']['symbol']
    fee_raw = int(pool['feeTier'])
    fee_percent = fee_raw / 10000
    symbol = f"{token0}_{token1}"
    fee_str = f"{fee_percent:.2f}pct"
    pool_id = pool['id']
    print(f"{i+1}. {symbol} (fee: {fee_percent:.2f}%) - Pool ID: {pool_id}")

    raw_data = fetch_pool_day_data(pool_id)
    df = process_data(raw_data, fee_raw)

    if df.empty:
        print(f"⚠️ Nenhum dado para {symbol}")
        continue

    filename = f"{output_dir}/{pool_id}_{token0}_{token1}_{fee_str}_last90d.csv"
    df.to_csv(filename, index=False)
    print(f"✅ Dados salvos: {filename}")

🔝 Top 5 Pools por volumeUSD:

1. USDC_WETH (fee: 0.05%) - Pool ID: 0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640
✅ Dados salvos: pool_data_csv/0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640_USDC_WETH_0.05pct_last90d.csv
2. WETH_USDT (fee: 0.05%) - Pool ID: 0x11b815efb8f581194ae79006d24e0d814b7697f6
✅ Dados salvos: pool_data_csv/0x11b815efb8f581194ae79006d24e0d814b7697f6_WETH_USDT_0.05pct_last90d.csv
3. USDC_USDT (fee: 0.01%) - Pool ID: 0x3416cf6c708da44db2624d63ea0aaef7113527c6
✅ Dados salvos: pool_data_csv/0x3416cf6c708da44db2624d63ea0aaef7113527c6_USDC_USDT_0.01pct_last90d.csv
4. WBTC_WETH (fee: 0.05%) - Pool ID: 0x4585fe77225b41b697c938b018e2ac67ac5a20c0
✅ Dados salvos: pool_data_csv/0x4585fe77225b41b697c938b018e2ac67ac5a20c0_WBTC_WETH_0.05pct_last90d.csv
5. USDC_WETH (fee: 0.30%) - Pool ID: 0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8
✅ Dados salvos: pool_data_csv/0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8_USDC_WETH_0.30pct_last90d.csv


# 📈 Seção 7. Visualização Interativa dos Dados Agregados

In [7]:
def plot_metrics_from_csv(csv_path):
    df = pd.read_csv(csv_path)
    df['date'] = pd.to_datetime(df['date'])
    pool_name = os.path.basename(csv_path).replace('_last90d.csv', '')

    metrics = [
        ('volumeUSD', '📈 Volume Diário'),
        ('tvlUSD', '🔐 TVL Diário'),
        ('feesUSD', '💸 Taxas Diárias'),
        ('dailyAPY', '📊 APY Diário Estimado'),
        ('liquidity', '🧊 Liquidez Total'),
        ('token0Price', '💱 Preço do Token0'),
        ('token1Price', '💱 Preço do Token1'),
    ]

    for col, title in metrics:
        if col in df.columns:
            fig = px.line(df, x='date', y=col, title=f'{title} - {pool_name}', markers=True)
            fig.update_layout(xaxis_title='Data', yaxis_title=col)
            fig.show()

# 📂 Seção 8. Ler e Plotar Todos os Arquivos CSV

In [8]:
csv_files = glob.glob(f"{output_dir}/*_last90d.csv")
if not csv_files:
    print("❌ Nenhum arquivo CSV encontrado.")
else:
    for csv_file in csv_files:
        plot_metrics_from_csv(csv_file)